In [ ]:
import numpy as np
import heapq # Importing Priority Queue

In [ ]:
# Initial State
init = np.array([
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
])
# Required Goal State
final = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
])
mode = 0

In [ ]:
# State class to store info about a state
class State:
    def __init__(self, state, x, y, cost, depth, parent = None):
        self.state = state
        self.x = x
        self.y = y
        self.cost = cost
        self.depth = depth
        self.parent = parent

    def __lt__(self, other):
        return (self.state < other.state).any()

In [ ]:
# Finding 0 in start state
start_x, start_y = np.where(np.array(init) == 0)
start = State(init, start_x[0], start_y[0], 0, 0)

In [ ]:
def find_cost(state, depth):
    displaced = 0
    manhattan = 0
    for i in range(1, 9):
        i_x, i_y = np.where(state == i)
        f_x, f_y = np.where(np.array(final) == i)
        manhattan += abs(i_x[0] - f_x[0]) + abs(i_y[0] - f_y[0])
        displaced += (i_x[0] != f_x[0] or i_y[0] != f_y[0])
    match mode:
        case 0:
            return (depth)
        case 1:
            return (displaced + depth)
        case 2:
            return (manhattan + depth)
        case 3:
            return (displaced + 2 * manhattan + depth)

In [ ]:
# Helper Functions for swapping in states
def swap_up(node):
    state = node.state.copy()
    x = node.x
    y = node.y
    state[x][y], state[x - 1][y] = state[x - 1][y], state[x][y]
    x = x - 1
    return State(state, x, y, find_cost(state, node.depth), node.depth + 1, node)

def swap_down(node):
    state = node.state.copy()
    x = node.x
    y = node.y
    state[x][y], state[x + 1][y] = state[x + 1][y], state[x][y]
    x = x + 1
    return State(state, x, y, find_cost(state, node.depth), node.depth + 1, node)

def swap_left(node):
    state = node.state.copy()
    x = node.x
    y = node.y
    state[x][y], state[x][y - 1] = state[x][y - 1], state[x][y]
    y = y - 1
    return State(state, x, y, find_cost(state, node.depth), node.depth + 1, node)

def swap_right(node):
    state = node.state.copy()
    x = node.x
    y = node.y
    state[x][y], state[x][y + 1] = state[x][y + 1], state[x][y]
    y = y + 1
    return State(state, x, y, find_cost(state, node.depth), node.depth + 1, node)

In [ ]:
# Helper Function for finding visited states
def find_in_open_closed(node, open, closed):
    for i in open:
        if (i[1].state == node.state).all() and i[1].cost > node.cost:
            i[1].cost = node.cost
            i[1].parent = node.parent
            return True

    pos = -1
    for i in range(len(closed)):
        if (closed[i].state == node.state).all():
            return True
            pos = i
            break
    if pos >= 0 and closed[i].cost > node.cost:
        n = closed.pop(i)
        n.cost = node.cost
        n.parent = node.parent
        open.append((n.cost, n))
        return True

    return False

In [ ]:
# Processing a state including finding its neighbours and adding them to open
def find_neighbours(node, open, closed):
    if node.x < 2:
        new_node = swap_down(node)
        if not find_in_open_closed(new_node, open, closed):
            heapq.heappush(open, (new_node.cost, new_node))

    if node.x > 0:
        new_node = swap_up(node)
        if not find_in_open_closed(new_node, open, closed):
            heapq.heappush(open, (new_node.cost, new_node))

    if node.y < 2:
        new_node = swap_right(node)
        if not find_in_open_closed(new_node, open, closed):
            heapq.heappush(open, (new_node.cost, new_node))

    if node.y > 0:
        new_node = swap_left(node)
        if not find_in_open_closed(new_node, open, closed):
            heapq.heappush(open, (new_node.cost, new_node))

In [ ]:
# Helper Function to print path from start to goal
def print_path(goal):
    path = [goal]
    while not (path[0].state == init).all():
        path.insert(0, path[0].parent)
    print(f"\n\tPath (Cost = {len(path) - 1}):- \n")
    for i in path:
        print(i.state, end = "\n\n")

In [ ]:
def astar():
    open = [(0, start)]  # Priority queue with (cost, state) tuples
    closed = []
    found_goal = False

    max_mem = 0 # save maximum memory usage

    while len(open):
        max_mem = max(max_mem, len(open) + len(closed))
        cost, n = heapq.heappop(open)
        if len(closed) > 2000 and len(open) > 2000:
            break
        # print(n.state, n.cost)
        closed.append(n)

        find_neighbours(n, open, closed)
        if (n.state == final).all(): # If goal is found, path is printed
            found_goal = True
            # print("Path", n.depth)
            print_path(n)
            break

    return {"Goal Found": found_goal != False, "Memory Usage (maximum states stored)": max_mem, "Time Taken (nodes visited)": len(closed)}

In [ ]:
mode = 0 # h(n) = 0
print(astar())

{'Goal Found': False, 'Memory Usage (maximum states stored)': 4995, 'Time Taken (nodes visited)': 2993}


In [ ]:
mode = 1 # h(n) = n(displaced tiles)
print(astar())

{'Goal Found': False, 'Memory Usage (maximum states stored)': 5270, 'Time Taken (nodes visited)': 3267}


In [ ]:
mode = 2 # h(n) = sum(manhattan distance)
print(astar())


	Path (Cost = 22):- 

[[0 1 2]
 [3 4 5]
 [6 7 8]]

[[1 0 2]
 [3 4 5]
 [6 7 8]]

[[1 4 2]
 [3 0 5]
 [6 7 8]]

[[1 4 2]
 [0 3 5]
 [6 7 8]]

[[1 4 2]
 [6 3 5]
 [0 7 8]]

[[1 4 2]
 [6 3 5]
 [7 0 8]]

[[1 4 2]
 [6 3 5]
 [7 8 0]]

[[1 4 2]
 [6 3 0]
 [7 8 5]]

[[1 4 2]
 [6 0 3]
 [7 8 5]]

[[1 4 2]
 [0 6 3]
 [7 8 5]]

[[1 4 2]
 [7 6 3]
 [0 8 5]]

[[1 4 2]
 [7 6 3]
 [8 0 5]]

[[1 4 2]
 [7 0 3]
 [8 6 5]]

[[1 0 2]
 [7 4 3]
 [8 6 5]]

[[1 2 0]
 [7 4 3]
 [8 6 5]]

[[1 2 3]
 [7 4 0]
 [8 6 5]]

[[1 2 3]
 [7 4 5]
 [8 6 0]]

[[1 2 3]
 [7 4 5]
 [8 0 6]]

[[1 2 3]
 [7 4 5]
 [0 8 6]]

[[1 2 3]
 [0 4 5]
 [7 8 6]]

[[1 2 3]
 [4 0 5]
 [7 8 6]]

[[1 2 3]
 [4 5 0]
 [7 8 6]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

{'Goal Found': True, 'Memory Usage (maximum states stored)': 1046, 'Time Taken (nodes visited)': 649}


In [ ]:
mode = 3 # h(n) = displaced + 2 * manhattan
print(astar())


	Path (Cost = 24):- 

[[0 1 2]
 [3 4 5]
 [6 7 8]]

[[3 1 2]
 [0 4 5]
 [6 7 8]]

[[3 1 2]
 [6 4 5]
 [0 7 8]]

[[3 1 2]
 [6 4 5]
 [7 0 8]]

[[3 1 2]
 [6 0 5]
 [7 4 8]]

[[3 1 2]
 [0 6 5]
 [7 4 8]]

[[0 1 2]
 [3 6 5]
 [7 4 8]]

[[1 0 2]
 [3 6 5]
 [7 4 8]]

[[1 2 0]
 [3 6 5]
 [7 4 8]]

[[1 2 5]
 [3 6 0]
 [7 4 8]]

[[1 2 5]
 [3 0 6]
 [7 4 8]]

[[1 2 5]
 [0 3 6]
 [7 4 8]]

[[0 2 5]
 [1 3 6]
 [7 4 8]]

[[2 0 5]
 [1 3 6]
 [7 4 8]]

[[2 3 5]
 [1 0 6]
 [7 4 8]]

[[2 3 5]
 [1 4 6]
 [7 0 8]]

[[2 3 5]
 [1 4 6]
 [7 8 0]]

[[2 3 5]
 [1 4 0]
 [7 8 6]]

[[2 3 0]
 [1 4 5]
 [7 8 6]]

[[2 0 3]
 [1 4 5]
 [7 8 6]]

[[0 2 3]
 [1 4 5]
 [7 8 6]]

[[1 2 3]
 [0 4 5]
 [7 8 6]]

[[1 2 3]
 [4 0 5]
 [7 8 6]]

[[1 2 3]
 [4 5 0]
 [7 8 6]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

{'Goal Found': True, 'Memory Usage (maximum states stored)': 819, 'Time Taken (nodes visited)': 487}


# Summary
## a) Mode 0 : h1(n) = 0  

## b) Mode 1 :  h2(n) = no of tiles displaced from  destined position

## c) Mode 2 :  h3(n) = sum of Manattan distance of each tile from goal position

## d) Mode 3 :  h3(n) = 2 * no of tiles displaced from  destined position